In [3]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import re 
from shutil import copyfile

In [34]:
parent = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_raw"
dirs = os.listdir(parent)

meta_data = []
for folder in dirs:
    xml_loc = glob.glob(parent + "/" + folder + "/*.xml")[0]
    tree = ET.parse(xml_loc)
    directory = [elem.text for elem in tree.iter() if "MASK_FILENAME" in elem.tag][0].split("/")[1]
    tile_id = directory.split("_")[1]
    filepath_partial = parent + "/" + directory + "/IMG_DATA" + "/R10m"
    filepath = glob.glob(filepath_partial + "/*.*")[0]
    filename = filepath.split("/")[-1]
    cloud_cover,no_data,unclassified = [elem.text for elem in tree.iter() if "CLOUDY_PIXEL_PERCENTAGE" in elem.tag 
             or "NODATA_PIXEL_PERCENTAGE" in elem.tag or "UNCLASSIFIED_PERCENTAGE" in elem.tag]
    meta_data.append([directory,tile_id,cloud_cover,no_data,unclassified,filename,filepath])
df = pd.DataFrame(meta_data,columns=["Directory","Tile_Id","Cloud Cover","No Data Percentage","Unclassified Percentage","Filename","Filepath"])




In [35]:
df

,Directory,Tile_Id,Cloud Cover,No Data Percentage,Unclassified Percentage,Filename,Filepath
0,L2A_T34MBC_A027031_20200825T092011,T34MBC,0.000000,99.778396,1.235196,T34MBC_20200825T085601_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
1,L2A_T34NEF_A026945_20200819T084315,T34NEF,7.542505,15.627228,6.340741,T34NEF_20200819T083611_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
2,L2A_T34NEF_A017965_20200814T085123,T34NEF,6.157278,16.400056,4.709803,T34NEF_20200814T083609_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
3,L2A_T34MBB_A026845_20200812T090428,T34MBB,0.016626,0.000000,42.896938,T34MBB_20200812T084601_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
4,L2A_T34MCB_A026845_20200812T090428,T34MCB,0.000000,0.000000,33.534166,T34MCB_20200812T084601_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
...,...,...,...,...,...,...,...
205,L2A_T33NUB_A010801_20190401T092807,T33NUB,14.963846,61.010295,4.361515,T33NUB_20190401T090559_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
206,L2A_T33NWB_A010801_20190401T092807,T33NWB,8.509690,0.000000,2.699487,T33NWB_20190401T090559_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
207,L2A_T33NUA_A010801_20190401T092807,T33NUA,19.550383,40.800050,5.260483,T33NUA_20190401T090559_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
208,L2A_T33NUC_A010801_20190401T092807,T33NUC,14.912106,81.202078,4.452609,T33NUC_20190401T090559_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...


In [65]:
#full_sorted df
df2 = df.sort_values(by=["Tile_Id","Unclassified Percentage"],ignore_index=True)

#partial df for testing
df_test = df2.head(50) 

In [69]:
df2.head()

,Directory,Tile_Id,Cloud Cover,No Data Percentage,Unclassified Percentage,Filename,Filepath
0,L2A_T33NTB_A017851_20200806T093251,T33NTB,14.723322,0.000000,20.213281,T33NTB_20200806T092029_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
1,L2A_T33NTB_A013990_20191110T093408,T33NTB,13.929234,0.000000,3.225338,T33NTB_20191110T092119_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
2,L2A_T33NTC_A023971_20200124T093439,T33NTC,20.687326,0.000000,1.377507,T33NTC_20200124T092251_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
3,L2A_T33NUA_A015091_20200126T091912,T33NUA,0.676950,41.920325,0.064221,T33NUA_20200126T091139_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
4,L2A_T33NUA_A015377_20200215T092112,T33NUA,1.286039,41.627392,0.288032,T33NUA_20200215T091019_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...


In [61]:
df.groupby("Tile_Id").apply(lambda x: x.sort_values("Unclassified Percentage")).head(20)

Directory Tile_Id Cloud Cover  \
Tile_Id                                                               
T33NTB  19   L2A_T33NTB_A017851_20200806T093251  T33NTB   14.723322   
        186  L2A_T33NTB_A013990_20191110T093408  T33NTB   13.929234   
T33NTC  148  L2A_T33NTC_A023971_20200124T093439  T33NTC   20.687326   
T33NUA  146  L2A_T33NUA_A015091_20200126T091912  T33NUA    0.676950   
        122  L2A_T33NUA_A015377_20200215T092112  T33NUA    1.286039   
        190  L2A_T33NUA_A022498_20191013T091824  T33NUA    3.900220   
        158  L2A_T33NUA_A023785_20200111T092435  T33NUA    6.707688   
        116  L2A_T33NUA_A024357_20200220T091348  T33NUA    1.893380   
        11   L2A_T33NUA_A026788_20200808T091204  T33NUA   12.003088   
        8    L2A_T33NUA_A026788_20200808T093036  T33NUA    1.135467   
        207  L2A_T33NUA_A010801_20190401T092807  T33NUA   19.550383   
T33NUB  125  L2A_T33NUB_A015377_20200215T092112  T33NUB    0.008738   
        200  L2A_T33NUB_A010844_20190404T093457  T33NUB    7.008033   
        168  L2A_T33NUB_A014705_20191230T093407  T33NUB   11.314472   
        161  L2A_T33NUB_A023785_20200111T092435  T33NUB   11.741149   
        205  L2A_T33NUB_A010801_20190401T092807  T33NUB   14.963846   
        198  L2A_T33NUB_A020925_20190625T091818  T33NUB   10.628747   
        119  L2A_T33NUB_A024357_20200220T091348  T33NUB   14.741878   
T33NUC  179  L2A_T33NUC_A014519_20191217T092124  T33NUC    0.154149   
        149  L2A_T33NUC_A023971_20200124T093439  T33NUC    1.880959   

            No Data Percentage Unclassified Percentage  \
Tile_Id                                                  
T33NTB  19            0.000000               20.213281   
        186           0.000000                3.225338   
T33NTC  148           0.000000                1.377507   
T33NUA  146          41.920325                0.064221   
        122          41.627392                0.288032   
        190          40.210217                1.319866   
        158          42.028204                1.463032   
        116          41.996250                1.559735   
        11           64.032459               14.158522   
        8            65.888530                2.706349   
        207          40.800050                5.260483   
T33NUB  125          61.841506                0.175332   
        200          20.374508                1.836948   
        168          19.600745               17.992981   
        161          62.230951                3.428186   
        205          61.010295                4.361515   
        198          61.122954                7.261848   
        119          62.171072                9.798358   
T33NUC  179          96.130061                0.204475   
        149           2.194084                1.252283   

                                       Filename  \
Tile_Id                                           
T33NTB  19   T33NTB_20200806T092029_TCI_10m.jp2   
        186  T33NTB_20191110T092119_TCI_10m.jp2   
T33NTC  148  T33NTC_20200124T092251_TCI_10m.jp2   
T33NUA  146  T33NUA_20200126T091139_TCI_10m.jp2   
        122  T33NUA_20200215T091019_TCI_10m.jp2   
        190  T33NUA_20191013T090921_TCI_10m.jp2   
        158  T33NUA_20200111T091341_TCI_10m.jp2   
        116  T33NUA_20200220T091011_TCI_10m.jp2   
        11   T33NUA_20200808T090601_TCI_10m.jp2   
        8    T33NUA_20200808T090601_TCI_10m.jp2   
        207  T33NUA_20190401T090559_TCI_10m.jp2   
T33NUB  125  T33NUB_20200215T091019_TCI_10m.jp2   
        200  T33NUB_20190404T092029_TCI_10m.jp2   
        168  T33NUB_20191230T092309_TCI_10m.jp2   
        161  T33NUB_20200111T091341_TCI_10m.jp2   
        205  T33NUB_20190401T090559_TCI_10m.jp2   
        198  T33NUB_20190625T090601_TCI_10m.jp2   
        119  T33NUB_20200220T091011_TCI_10m.jp2   
T33NUC  179  T33NUC_20191217T091309_TCI_10m.jp2   
        149  T33NUC_20200124T092251_TCI_10m.jp2   

                                                      Filepath  
Tile_Id   

In [45]:
df2["Tile_Id"].value_counts()

T33NWA    10
T34MCD    10
T34NDF    10
T33NUC     9
Name: Tile_Id, dtype: int64

In [47]:
base_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/"
layer = 1
df = df2
for index,row in df.iterrows(): 
    destination_dir = base_dir + str(layer)
    print("destination_dir:",destination_dir)
    output_file = destination_dir + "/" + row["Filename"]
    print("output_file:",output_file)
    
    # Check if directory exists
    if not os.path.isdir(destination_dir):
        os.mkdir(destination_dir)
        
    # Copy file to existing or new directory
    copyfile(row["Filepath"],output_file)
    
    # Check if Tile_Id already exists in the directory - only necessary up until the last tile
    if len(df) > index + 1:
        if df.loc[index,"Tile_Id"] == df.loc[index + 1,"Tile_Id"]:
            layer += 1
        else:
            layer = 1   

destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/1
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/1/T33NUC_20191217T091309_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/2
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/2/T33NUC_20190404T092029_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/3
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/3/T33NUC_20191217T091309_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/Ordered/4
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1

In [1]:
# for row in range(len(df5)):
#     layer = 1
#     dest_folder = f'.../layer_{layel}'
#     # Copy TCI to dest_folder
#     if not os.exists(dest_folder):
#         os.mkdir(dest_folder)

#     if row != len(df5)-1:
#         if df5.loc[row, 'tileid_new'] == df5.loc[row+1, 'tileid_new']:
#             layer += 1
#         else:
#             layer = 1

# For Finding Corresponding SCL File for each tile from SAFE_true

In [28]:
source_dir_SAFE_true = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_raw/"
source_dir_SAFE_false = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_false_raw/"

xml_source = glob.glob(source_dir_SAFE_true + "*/*.xml")[0]

print(os.listdir(source_dir_SAFE_false)[0],os.listdir(source_dir_SAFE_true)[0])

SAFE_false_dir = os.listdir(source_dir_SAFE_false)[0]
SAFE_true_dir = os.listdir(source_dir_SAFE_true)

34MBC,2020-08-25,0 L2A_T34MBC_A027031_20200825T092011


In [25]:
tile_id = SAFE_false_dir.split(",")[0]
tile_date = SAFE_false_dir.split(",")[1]

In [32]:
SAFE_true_dir[0].split("_")[1]

'T34MBC'